# Create arXiv Embeddings

In this notebook we will:

1) Pull the arXiv dataset from Kaggle
2) Perform data preprocessing and cleanup
3) Create HuggingFace embeddings
4) Create OpenAI embeddings

In [6]:
!rfd

zsh:1: command not found: rfd


In [3]:
import kaggle

In [4]:
!kaggle datasets download -d Cornell-University/arxiv

Dataset URL: https://www.kaggle.com/datasets/Cornell-University/arxiv
License(s): CC0-1.0
arxiv.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzip the file and there you have it!

## 2 - Perform data preprocessing and cleanup

In [7]:
import json
import pandas as pd
import re
import string


DATA_PATH = "arxiv-metadata-oai-snapshot.json"
YEAR_CUTOFF = 2012
YEAR_PATTERN = r"(19|20[0-9]{2})"
ML_CATEGORY = "cs.LG"
DATASET_SIZE=10000

In [8]:
# Preprocessing function to clean data
def process(paper: dict):
    paper = json.loads(paper)
    if paper['journal-ref']:
        years = [int(year) for year in re.findall(YEAR_PATTERN, paper['journal-ref'])]
        years = [year for year in years if (year <= 2023 and year >= 1991)]
        year = min(years) if years else None
    else:
        year = None
    return {
        'id': paper['id'],
        'title': paper['title'],
        'year': year,
        'authors': paper['authors'],
        'categories': ','.join(paper['categories'].split(' ')),
        'abstract': paper['abstract']
    }

# Data loading function
def papers():
    with open(DATA_PATH, 'r') as f:
        for paper in f:
            paper = process(paper)
            if paper['year']:
                if paper['year'] >= YEAR_CUTOFF and ML_CATEGORY in paper['categories']:
                    yield paper

In [9]:
# Load dataset into Pandas dataframe and take a sample
df = pd.DataFrame(papers()).sample(n=DATASET_SIZE)

In [10]:
# Avg length of the abstracts - num tokens
df.abstract.apply(lambda a: len(a.split())).mean()

172.4068

In [11]:
# Helper function to clean the description!
def clean_description(description: str):
    if not description:
        return ""
    # remove unicode characters
    description = description.encode('ascii', 'ignore').decode()

    # remove punctuation
    description = re.sub('[%s]' % re.escape(string.punctuation), ' ', description)

    # clean up the spacing
    description = re.sub('\s{2,}', " ", description)

    # remove urls
    #description = re.sub("https*\S+", " ", description)

    # remove newlines
    description = description.replace("\n", " ")

    # remove all numbers
    #description = re.sub('\w*\d+\w*', '', description)

    # split on capitalized words
    description = " ".join(re.split('(?=[A-Z])', description))

    # clean up the spacing again
    description = re.sub('\s{2,}', " ", description)

    # make all words lowercase
    description = description.lower()

    return description.strip()

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
/var/folders/mk/41lqkkzj29d7jlzz09x5gb300000gp/T/ipykernel_38526/1561391420.py:12: SyntaxWarning: invalid escape sequence '\s'
  description = re.sub('\s{2,}', " ", description)
/var/folders/mk/41lqkkzj29d7jlzz09x5gb300000gp/T/ipykernel_38526/1561391420.py:27: SyntaxWarning: invalid escape sequence '\s'
  description = re.sub('\s{2,}', " ", description)


In [12]:
# Apply the cleaner method on both title and abstract
texts = df.apply(lambda r: clean_description(r['title'] + ' ' + r['abstract']), axis=1).tolist()

## 3 - Creating Hugging Face Embeddings

First up, we will use the built-in RedisVL vectorizer to create embeddings from huggingface.

In [13]:
!pipx install redisvl==0.1.0
!pipx install transformers
!pipx install sentence-transformers --include-deps

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'redisvl' already seems to be installed. Not modifying existing installation
in '/Users/rouzbeh.farahmand/.local/pipx/venvs/redisvl'. Pass '--force' to
force installation.
'transformers' already seems to be installed. Not modifying existing
installation in '/Users/rouzbeh.farahmand/.local/pipx/venvs/transformers'.
Pass '--force' to force installation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'sentence-transformers' already seems to be installed. Not modifying existing
installation in
'/Users/rouzbeh.farahmand/.local/pipx/venvs/sentence-transformers'. Pass
'--force' to force installation.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
!pipx ensurepath

/Users/rouzbeh.farahmand/.local/bin has been been added to PATH, but you need
    to open a new terminal or re-login for this PATH change to take effect.

You will need to open a new terminal or re-login for the PATH changes to take
effect.

Otherwise pipx is ready to go! ✨ 🌟 ✨


In [1]:
!exec /bin/zsh

rouzbeh.farahmand@rouzbeh data %                                           
rouzbeh.farahmand@rouzbeh data %                                   
rouzbeh.farahmand@rouzbeh data %                                           

In [4]:
!pip3


Usage:   
  pip3 <command> [options]

Commands:
  install                     Install packages.
  download                    Download packages.
  uninstall                   Uninstall packages.
  freeze                      Output installed packages in requirements format.
  inspect                     Inspect the python environment.
  list                        List installed packages.
  show                        Show information about installed packages.
  check                       Verify installed packages have compatible dependencies.
  config                      Manage local and global configuration.
  search                      Search PyPI for packages.
  cache                       Inspect and manage pip's wheel cache.
  index                       Inspect information available from package indexes.
  wheel                       Build wheels from your requirements.
  hash                        Compute hashes of package archives.
  completion                  A helper c

In [1]:
!pip install sentence-transformers

zsh:1: command not found: pip


In [22]:
from sentence_transformers import SentenceTransformer

In [14]:
from redisvl.utils.vectorize import HFTextVectorizer

hf = HFTextVectorizer(model="sentence-transformers/all-mpnet-base-v2")

In [15]:
# Create embeddings from the title and abstract
embeddings = hf.embed_many(texts)

In [16]:
embeddings[0][:10]

[-0.014411247335374355,
 0.01784348115324974,
 -0.006457224488258362,
 0.044496506452560425,
 -0.015901479870080948,
 0.012798929587006569,
 0.028422124683856964,
 -0.038135871291160583,
 0.025910435244441032,
 -0.0353618748486042]

In [18]:
# Add embeddings to df
df = df.reset_index().drop('index', axis=1)
df['huggingface'] = embeddings

In [19]:
df['cohere'] = embeddings

In [20]:
df['openai'] = embeddings

## OpenAI Embeddings

Next, we will use OpenAI Embeddings for our arXiv papers. You will need to set your OpenAI API Key below!

In [16]:
from redisvl.utils.vectorize import OpenAITextVectorizer

oai = OpenAITextVectorizer(api_config={"api_key": "YOUR API KEY HERE"})

embeddings = await oai.aembed_many(texts)

In [17]:
len(embeddings)

1000

In [18]:
embeddings[0][:10]

[-0.034895237535238266,
 0.0013622459955513477,
 0.025790950283408165,
 -0.031307876110076904,
 -0.0186705831438303,
 0.027937931939959526,
 0.008866488933563232,
 0.01263049989938736,
 -0.03155247122049332,
 -0.022502535954117775]

In [19]:
df['openai'] = embeddings

## Cohere Embeddings

In [20]:
from redisvl.utils.vectorize import CohereTextVectorizer

co = CohereTextVectorizer(
    model="embed-multilingual-v3.0",
    api_config={"api_key": "YOUR API KEY HERE"}
)

In [22]:
embeddings = co.embed_many(texts, input_type="search_document")

In [23]:
len(embeddings)

1000

In [24]:
len(embeddings[0])

1024

In [25]:
df['cohere'] = embeddings

In [26]:
df.head()

,id,title,year,authors,categories,abstract,huggingface,openai,cohere
0,1812.02855,Progressive Sampling-Based Bayesian Optimizati...,2017,"Xueqiang Zeng, Gang Luo","cs.LG,stat.ML",Purpose: Machine learning is broadly used fo...,"[0.005483315791934729, 0.06285043805837631, -0...","[-0.034895237535238266, 0.0013622459955513477,...","[-0.0146865845, -0.023620605, 0.009109497, 0.0..."
1,1708.01422,Exploring the Function Space of Deep-Learning ...,2018,Bo Li and David Saad,"cond-mat.dis-nn,cs.LG",The function space of deep-learning machines...,"[-0.022667571902275085, 0.04551266133785248, -...","[-0.017496585845947266, -0.009123609401285648,...","[0.020828247, -0.004623413, -0.009017944, 0.06..."
2,2001.00561,PrivacyNet: Semi-Adversarial Networks for Mult...,2020,"Vahid Mirjalili, Sebastian Raschka, Arun Ross","cs.CV,cs.CR,cs.LG",Recent research has established the possibil...,"[-0.004289142321795225, 0.1055050864815712, -0...","[-0.0183021891862154, 0.004181693773716688, 0....","[0.024093628, 0.0047302246, -0.032104492, 0.04..."
3,2003.01300,Few-Shot Relation Learning with Attention for ...,2020,"Sion An, Soopil Kim, Philip Chikontwe and Sang...","eess.SP,cs.LG",Brain-Computer Interfaces (BCI) based on Ele...,"[-0.033013615757226944, 0.05156606808304787, -...","[-0.036494333297014236, 0.003500517923384905, ...","[-0.045196533, -0.009727478, 0.016662598, 0.00..."
4,1902.03896,Reconstructing dynamical networks via feature ...,2019,Marc G. Leguia and Zoran Levnajic and Ljupco T...,"math.DS,cs.LG,cs.SI,physics.soc-ph,stat.ML",Empirical data on real complex systems are b...,"[-0.07013913244009018, 0.05345052108168602, -0...","[-0.017974110320210457, 0.010659225285053253, ...","[0.05041504, 0.03857422, -0.00983429, 0.029983..."


In [22]:
d = df.to_dict('records')

In [23]:
import json

# Write to file

with open("arxiv-papers-10000.json", "w") as f:
    json.dump(d, f)